# Tuning experiments for the Movielens dataset

Feel free to re-run this and change hyperparameters as you see fit.

This notebook should also be a good place to load the data only once and then train multiple times, generate charts, etc.


In [1]:
from data_loader import load_data, train_test_split, MovielensDataset
import numpy as np
import torch
from torch import nn
from torch.optim import Adam
import argparse
import yaml
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error

from gmf import GMF
from ncf_mlp import NCF_MLP
from neural_mf import NEURAL_MF

from train import train_gmf, train_joint_nerual_mf, train_mlp, train_neural_mf, train_loop, test_loop

In [27]:
# Starting hyperparameters
learning_rate = 0.05
weight_decay = 0.0000001
epochs = 5
batch_size = 1024
latent_dims = 16
alpha: 0.5

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [29]:
# Load the data
dataset = '100k'
data, num_users, num_items = load_data(dataset)
train, test = train_test_split(data)

In [30]:
train = MovielensDataset(users=train['user_id'], movies=train['movie_id'], ratings = train['rating'])
test = MovielensDataset(users=test['user_id'], movies=test['movie_id'], ratings = test['rating'])

train_dataloader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test, batch_size=batch_size, shuffle=True, num_workers= 4)

## Tune Hyperparameters for Models

In [31]:
# Tune some hyperparameters for GMF
def tune_lr(model_type='gmf'):
    learning_rates = np.array([0.001, 0.01, 0.05, 0.07, 0.1, 0.2])
    rmse_history = []
    best_model = None
    best_lr = 0.01
    best_RMSE = np.inf
    for lr in learning_rates:
        if model_type == 'gmf':
            print('TRAINING GMF')
            train_model = train_gmf
        elif model_type == 'mlp':
            print('TRAINING MLP')
            train_model = train_mlp
        else:
            print('TRAINING NMF')
            train_model = train_nmf
        model = train_model(train_dataloader, test_dataloader, num_users, num_items, epochs, latent_dims,
                            lr, # Param being tuned
                            'adam', criterion=nn.CrossEntropyLoss(), device=device, weight_decay=weight_decay)

        # Select the best model based on TRAIN RMSE, not TEST RMSE
        rmse = test_loop(train_dataloader, model, nn.CrossEntropyLoss(), device)
        rmse_history.append(rmse)

        if rmse < best_RMSE:
            best_lr = lr
            best_RMSE = rmse
            best_model = model

    rmse_history = np.array(rmse_history)

    print()
    print('Finished tuning model', model_type)
    print('Best Learning Rate:', best_lr)
    print('Best RMSE:', best_RMSE)

    return (learning_rates, rmse_history, best_model)


In [32]:
gmf_lr, gmf_rmse, opt_model = tune_lr('gmf')
# TODO: Plot this
print(gmf_lr)
print(gmf_rmse)

TRAINING GMF
Epoch 1
------------------------
loss: 4924.128418  [ 1024/79619]
Test MSE 0.08242572277956661
Test RMSE 0.2870988031663779
Epoch 2
------------------------
loss: 5020.475098  [ 1024/79619]
Test MSE 0.0736839597842501
Test RMSE 0.27144789515531353
Epoch 3
------------------------
loss: 5034.125977  [ 1024/79619]
Test MSE 0.07028287796379766
Test RMSE 0.26510918121369853
Epoch 4
------------------------
loss: 4990.594238  [ 1024/79619]
Test MSE 0.06921984075217374
Test RMSE 0.26309663766793706
Epoch 5
------------------------
loss: 5015.585938  [ 1024/79619]
Test MSE 0.06892058711736244
Test RMSE 0.262527307374609
Test MSE 0.06801088713490124
Test RMSE 0.26078897050086536
TRAINING GMF
Epoch 1
------------------------
loss: 5114.275391  [ 1024/79619]
Test MSE 0.1278021387128007
Test RMSE 0.3574942499017302
Epoch 2
------------------------
loss: 5040.489258  [ 1024/79619]
Test MSE 0.163598129205617
Test RMSE 0.40447265569580476
Epoch 3
------------------------
loss: 5011.6083

In [35]:
# Tune some hyperparameters for GMF
def tune_dims(model_type='gmf'):
    dims = [8, 16, 32, 64, 128, 256]
    rmse_history = []
    best_model = None
    best_dim = 0.01
    best_RMSE = np.inf
    for dim in dims:
        if model_type == 'gmf':
            print('TRAINING GMF')
            train_model = train_gmf
        elif model_type == 'mlp':
            print('TRAINING MLP')
            train_model = train_mlp
        else:
            print('TRAINING NMF')
            train_model = train_nmf
        model = train_model(train_dataloader, test_dataloader, num_users, num_items, epochs,
                            dim, # Param being tuned 
                            learning_rate, 'adam', criterion=nn.CrossEntropyLoss(), device=device, weight_decay=weight_decay)

        # Select the best model based on TRAIN RMSE, not TEST RMSE
        rmse = test_loop(train_dataloader, model, nn.CrossEntropyLoss(), device)
        rmse_history.append(rmse)

        if rmse < best_RMSE:
            best_dim = dim
            best_RMSE = rmse
            best_model = model

    rmse_history = np.array(rmse_history)

    print()
    print('Finished tuning model', model_type)
    print('Best Embedding Dim:', best_dim)
    print('Best RMSE:', best_RMSE)

    return (dims, rmse_history, best_model)


In [36]:
gmf_dim, gmf_rmse, opt_model = tune_dims('gmf')
# TODO: Plot this
print(gmf_dim)
print(gmf_rmse)


TRAINING GMF
Epoch 1
------------------------
loss: 5021.759766  [ 1024/79619]
Test MSE 0.4614989715264173
Test RMSE 0.6793371560031273
Epoch 2
------------------------
loss: 4994.810547  [ 1024/79619]
Test MSE 0.4836110871890738
Test RMSE 0.6954215176345019
Epoch 3
------------------------
loss: 5070.592773  [ 1024/79619]
Test MSE 0.2567314553289608
Test RMSE 0.5066867427996916
Epoch 4
------------------------
loss: 5000.495117  [ 1024/79619]
Test MSE 0.04493780511762326
Test RMSE 0.21198538892485788
Epoch 5
------------------------
loss: 5058.890625  [ 1024/79619]
Test MSE 0.044639099556100105
Test RMSE 0.21127967142179133
Test MSE 0.030248094296741072
Test RMSE 0.1739197927112986
TRAINING GMF
Epoch 1
------------------------
loss: 5071.441895  [ 1024/79619]
Test MSE 0.4418403193317014
Test RMSE 0.6647107034881427
Epoch 2
------------------------
loss: 4963.020508  [ 1024/79619]
Test MSE 0.44970225380123363
Test RMSE 0.670598429614351
Epoch 3
------------------------
loss: 5016.37841